

<img src="https://user-images.githubusercontent.com/7065401/39117173-a433bf6a-46e6-11e8-8a40-b4d4d6422493.jpg"
    style="width:300px; float: right; margin: 0 40px 40px 40px;"></img>

# Limpiando valores no nulos

Después de tratar con muchos conjuntos de datos, puedo decirte que los "datos faltantes" no son tan importantes. Lo mejor que puede pasar es ver claramente valores como `np.nan`. Lo único que tenes que hacer es utilizar métodos como `isnull` y `fillna`/`dropna` y pandas se encargará del resto.Pero a veces, puedes tener valores no válidos que no son simplemente "datos faltantes" (`None`, o `nan`). Por ejemplo:




![separator2](https://i.imgur.com/4gX5WFr.png)



In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.DataFrame({
    'Sex': ['M', 'F', 'F', 'D', '?'],
    'Age': [29, 30, 24, 290, 25],
})
df

,Sex,Age
0,M,29
1,F,30
2,F,24
3,D,290
4,?,25


El `DataFrame` anterior no tiene ningún "valor faltante", pero claramente tiene datos no válidos. La edad '290' no parece ser una edad válida, y 'D' y '?' no se corresponden con ninguna categoría de sexo conocida. Entonces, ¿cómo se pueden limpiar estos valores que no faltan, pero que son claramente inválidos?

### Buscando valores unicos

El primer paso para limpiar valores invalidos es encontrarlos o identificarlos, para luego eliminarlos o reemplazarlos. Por lo general, para un campo categorico donde las posibilidades son discretas comenzamos analizando la variedad de los valores presentes. Para esto usamos el método `unique()`.


In [3]:
df['Sex'].unique()

array(['M', 'F', 'D', '?'], dtype=object)

In [4]:
df['Sex'].value_counts()

,count
Sex,
F,2
M,1
D,1
?,1


Claramente, si ves valores como `'D'` o `'?'`, inmediatamente llamará tu atención. Ahora bien, ¿qué hacer con ellos? Vamos a suponer que te dicen que `'D'` fue un error tipográfico y que en realidad debería ser `'F'`. Podemos utilizar la función `replace` para reemplazar estos valores:



In [ ]:
df['Sex'].replace('D', 'F')

0    M
1    F
2    F
3    F
4    ?
Name: Sex, dtype: object

La función `replace` puede aceptar un diccionario de valores para reemplazar. Por ejemplo, también te dijeron que podría haber algunas `'N's`, que en realidad deberían ser `'M's`:


In [ ]:
df['Sex'].replace({'D': 'F', 'N': 'M'})

0    M
1    F
2    F
3    F
4    ?
Name: Sex, dtype: object

Si hay mas de una columna con valores a reemplazar se puede aplicar a nivel de "DataFrame":

In [ ]:
df.replace({
    'Sex': {
        'D': 'F',
        'N': 'M'
    },
    'Age': {
        290: 29
    }
})

,Sex,Age
0,M,29
1,F,30
2,F,24
3,F,29
4,?,25


En el ejemplo anterior, reemplazamos explícitamente 290 por 29 (asumiendo que era solo un 0 adicional ingresado en la fase de entrada de datos). Pero, ¿qué pasa si desea eliminar todos los 0 adicionales de las columnas de edades? (ejemplo, `150 > 15`, `490 > 49`).El primer paso sería establecer el límite de la edad. ¿Es 100? 120? Digamos que cualquier cosa por encima de 100 no es creíble para **nuestro** conjunto de datos. A continuación, podemos combinar la selección booleana con la operación:





In [ ]:
df[df['Age'] > 100]

,Sex,Age
3,D,290


y dividimos por 10:

In [6]:
df.loc[df['Age'] > 100, 'Age'] = df.loc[df['Age'] > 100, 'Age'] / 10

# df['Age'] > 100 Da True donde se cumple la condicion
# df['Age'] > 100, 'Age'] / 10   Para los casos True toma el valor de age y lo divide por 10 y lo almacena en la misma posicion


In [ ]:
df

,Sex,Age
0,M,29.0
1,F,30.0
2,F,24.0
3,D,29.0
4,?,25.0


In [ ]:
df['Age'] = df['Age'].astype(int)

In [ ]:
df

,Sex,Age
0,M,29
1,F,30
2,F,24
3,D,290
4,?,25


![separator1](https://i.imgur.com/ZUWYTii.png)

### Duplicados

Comprobar la existencia de valores duplicados es extremadamente sencillo. Se comporta de manera diferente entre Series y DataFrames. Empecemos por las series. Por ejemplo, supongamos que estamos organizando una fiesta elegante e invitamos a embajadores de Europa. Pero solo se puede invitar a un embajador por país. Esta es nuestra lista original, y como puedes ver, tanto el Reino Unido como Alemania tienen embajadores duplicados:



In [ ]:
ambassadors = pd.Series([
    'France',
    'United Kingdom',
    'United Kingdom',
    'Italy',
    'Germany',
    'Germany',
    'Germany',
], index=[
    'Gérard Araud',
    'Kim Darroch',
    'Peter Westmacott',
    'Armando Varricchio',
    'Peter Wittig',
    'Peter Ammon',
    'Klaus Scharioth '
])

In [ ]:
ambassadors

,0
Gérard Araud,France
Kim Darroch,United Kingdom
Peter Westmacott,United Kingdom
Armando Varricchio,Italy
Peter Wittig,Germany
Peter Ammon,Germany
Klaus Scharioth,Germany


El método `duplicated` (nos dice quer valores están duplicados) y `drop_duplicates` (eliminara los duplicados):

In [ ]:
ambassadors.duplicated()

,0
Gérard Araud,False
Kim Darroch,False
Peter Westmacott,True
Armando Varricchio,False
Peter Wittig,False
Peter Ammon,True
Klaus Scharioth,True


En este caso `duplicated` no considera a `'Kim Darroch'`, la primer ocurrencia para United Kingdom o a `'Peter Wittig'` como duplicados. Esto es, ya que por defecto, considera la primer ocurrencia como no duplicada. Se puede cambiar este comportamiento con el parametro `keep`:

In [ ]:
ambassadors.duplicated(keep='last')

,0
Gérard Araud,False
Kim Darroch,True
Peter Westmacott,False
Armando Varricchio,False
Peter Wittig,True
Peter Ammon,True
Klaus Scharioth,False


Ahora, `'Kim Darroch'` y `'Peter Wittig'` (la primer ocurrencia de cada país) es considerada duplicada, pero `'Peter Westmacott'` y `'Klaus Scharioth'` no son considerados duplicados. También se puede seleccionar para que todos sean considerados duplicados con el valor de `keep=False`:

In [ ]:
ambassadors.duplicated(keep=False)

Gérard Araud          False
Kim Darroch            True
Peter Westmacott       True
Armando Varricchio    False
Peter Wittig           True
Peter Ammon            True
Klaus Scharioth        True
dtype: bool

El método `drop_duplicates`, elimina los valores duplicados y tiene el parametro `keep` visto en el ejemplo anterior:

In [ ]:
ambassadors.drop_duplicates()

Gérard Araud                  France
Kim Darroch           United Kingdom
Armando Varricchio             Italy
Peter Wittig                 Germany
dtype: object

In [ ]:
ambassadors.drop_duplicates(keep='last')

Gérard Araud                  France
Peter Westmacott      United Kingdom
Armando Varricchio             Italy
Klaus Scharioth              Germany
dtype: object

In [ ]:
ambassadors.drop_duplicates(keep=False)

Gérard Araud          France
Armando Varricchio     Italy
dtype: object

### Duplicados en DataFrames

Conceptualmente hablando, los duplicados en un DataFrame ocurren en el nivel de "fila". Dos filas con exactamente los mismos valores se consideran duplicados:

In [ ]:
players = pd.DataFrame({
    'Name': [
        'Kobe Bryant',
        'LeBron James',
        'Kobe Bryant',
        'Carmelo Anthony',
        'Kobe Bryant',
    ],
    'Pos': [
        'SG',
        'SF',
        'SG',
        'SF',
        'SF'
    ]
})

In [ ]:
players

,Name,Pos
0,Kobe Bryant,SG
1,LeBron James,SF
2,Kobe Bryant,SG
3,Carmelo Anthony,SF
4,Kobe Bryant,SF


En el DataFrame anterior, vemos claramente que Kobe está duplicado; Pero aparece con dos posiciones diferentes. Veamos la salida del método `duplicated`

In [ ]:
players.duplicated()

,0
0,False
1,False
2,True
3,False
4,False


El método `duplicated` devuelve una serie booleana que indica si una fila es un duplicado de una fila anterior en el DataFrame (basándose en todas las columnas por defecto). Solo la primera ocurrencia de cada conjunto duplicado se marca como False, y las repeticiones posteriores como True.

Para buscar duplicados en una columna especifica, usamos el parámetro `subset`:

In [ ]:
players.duplicated(subset=['Name'])

,0
0,False
1,False
2,True
3,False
4,True


y aplican las mismas reglas para el parámetro `keep`:

In [ ]:
players.duplicated(subset=['Name'], keep='last')

`drop_duplicates` takes the same parameters:

In [ ]:
players.drop_duplicates()

In [ ]:
players.drop_duplicates(subset=['Name'])

In [ ]:
players.drop_duplicates(subset=['Name'], keep='last')

![separator1](https://i.imgur.com/ZUWYTii.png)

### Normalizando Texto

Limpiar los valores de texto puede ser increíblemente difícil. Los valores de texto no válidos implican, el 99% de las veces, errores de escritura, que son completamente impredecibles y no siguen ningún patrón. Afortunadamente, no es tan común en estos días, donde las tareas de entrada de datos han sido reemplazadas por máquinas. Aún así, exploremos los casos más comunes:

### Separando Columnas

Vamos a suponer que cargamos un DF con los resultados de una encuesta:

In [7]:
df = pd.DataFrame({
    'Data': [
        '1987_M_US _1',
        '1990?_M_UK_1',
        '1992_F_US_2',
        '1970?_M_   IT_1',
        '1985_F_I  T_2'
]})

In [8]:
df

,Data
0,1987_M_US _1
1,1990?_M_UK_1
2,1992_F_US_2
3,1970?_M_ IT_1
4,1985_F_I T_2


Sabemos que las columnas individuales representan los valores "año, sexo, país y número de hijos", pero todo se ha agrupado en la misma columna y se ha separado por un guión bajo. Pandas tiene un método conveniente llamado `split` que podemos usar en estas situaciones:

In [9]:
df['Data'].str.split('_')

,Data
0,"[1987, M, US , 1]"
1,"[1990?, M, UK, 1]"
2,"[1992, F, US, 2]"
3,"[1970?, M, IT, 1]"
4,"[1985, F, I T, 2]"


In [10]:
df['Data'].str.split('_', expand=True)   # Expandimos en columnas

,0,1,2,3
0,1987,M,US,1
1,1990?,M,UK,1
2,1992,F,US,2
3,1970?,M,IT,1
4,1985,F,I T,2


In [ ]:
df = df['Data'].str.split('_', expand=True)    # Sin expand true serie de listas, con true expande a columnas

In [ ]:
df.columns = ['Year', 'Sex', 'Country', 'No Children']

Podemos buscar valores con el metodos `contains`:

In [ ]:
df

,Year,Sex,Country,No Children
0,1987,M,US,1
1,1990?,M,UK,1
2,1992,F,US,2
3,1970?,M,IT,1
4,1985,F,I T,2


In [ ]:
df['Year'].str.contains('\?')

0    False
1     True
2    False
3     True
4    False
Name: Year, dtype: bool

Como [`contains`](http://pandas.pydata.org/pandas-docs/version/0.22.0/generated/pandas.Series.str.contains.html) puede tomar una expresión regular (regex/pattern) com primer valor es necesario poner la barra antes del simbolo `?` para especificar que es el caracter y no una expresion regular. Para las letras no es necesario poner el caracter de escape:

In [ ]:
df['Country'].str.contains('U')

0     True
1     True
2     True
3    False
4    False
Name: Country, dtype: bool

Sacando espacios en blanco (como en `'US '` o `'I  T'` se puede hacer con los metodos `strip` (`lstrip` y `rstrip` tambien existen) o simplementwe con `replace`:

In [ ]:
df['Country'].str.strip()

,Country
0,US
1,UK
2,US
3,IT
4,I T


In [ ]:
df['Country'].str.replace(' ', '')

,Country
0,US
1,UK
2,US
3,IT
4,IT


Como se puede ver, todas estas operaciones relacionadas con cadenas/texto se aplican sobre el atributo `str` de la serie. Para mas información les dejo un link de consulta [aca](https://pandas.pydata.org/pandas-docs/stable/text.html).

![separator2](https://i.imgur.com/4gX5WFr.png)